In [1]:
#wordnet
import csv
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn

# Download NLTK resources if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    # Tokenize the text into words
    words = word_tokenize(text)

    # Convert words to lowercase
    words = [word.lower() for word in words]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    return words

def get_sentiment_score(word, words):
    synsets = wn.synsets(word)
    if synsets:
        # Check if there is any positive or negative sentiment in the synsets
        for synset in synsets:
            # Check if the synset contains any positive or negative word
            for lemma in synset.lemmas():
                if lemma.antonyms():  # If word has an antonym, it's opposite in sentiment
                    antonyms = lemma.antonyms()
                    # Check if the antonym is in the sentence
                    for antonym in antonyms:
                        if antonym.name() in words:
                            return -1  # Negative sentiment due to negation
                elif lemma.name() == word:
                    return 1  # Positive sentiment (if the word is a direct synonym)
    return 0  # Neutral sentiment

def analyze_sentiment(text):
    # Preprocess the text
    words = preprocess_text(text)

    # Initialize variables to count sentiment scores
    positive_scores = []
    negative_scores = []
    word_sentiment_scores = {}

    # Analyze sentiment scores for each word
    for word in words:
        # Calculate sentiment scores using WordNet
        sentiment_score = get_sentiment_score(word, words)

        # Store sentiment score for each word
        word_sentiment_scores[word] = sentiment_score

        # Update positive and negative scores
        if sentiment_score > 0:
            positive_scores.append(sentiment_score)
        elif sentiment_score < 0:
            negative_scores.append(sentiment_score)

    # Calculate average positive and negative scores
    if positive_scores:
        avg_positive_score = sum(positive_scores) / len(positive_scores)
    else:
        avg_positive_score = 0

    if negative_scores:
        avg_negative_score = sum(negative_scores) / len(negative_scores)
    else:
        avg_negative_score = 0

    # Calculate overall sentiment score for the sentence
    overall_sentiment_score = avg_positive_score - avg_negative_score

    return avg_positive_score, avg_negative_score, overall_sentiment_score

# Example data
data = [
    {"author_id": 1, "text": "The movie was not good. It was actually quite bad."},
    {"author_id": 2, "text": "I love this book! It's fantastic."},
    {"author_id": 3, "text": "The weather today is terrible."},
    {"author_id": 4, "text": "The concert was amazing."},
]

# Write sentiment analysis results to a CSV file
with open("sentiment_analysis_results.csv", "w", newline="") as csvfile:
    fieldnames = ["author_id", "text", "avg_positive_score", "avg_negative_score", "overall_sentiment_score"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    for entry in data:
        author_id = entry["author_id"]
        text = entry["text"]

        # Perform sentiment analysis
        avg_positive_score, avg_negative_score, overall_sentiment_score = analyze_sentiment(text)

        # Write to CSV
        writer.writerow({
            "author_id": author_id,
            "text": text,
            "avg_positive_score": avg_positive_score,
            "avg_negative_score": avg_negative_score,
            "overall_sentiment_score": overall_sentiment_score
        })

print("CSV file created successfully!")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


CSV file created successfully!
